In [6]:
import pandas as pd
import numpy as np
import os

print("=== ELABORAZIONE FEATURES AUDIO - DATASET DEAM ===")
print("=" * 50)

# Definisci il percorso alla cartella delle features
features_dir = '../data/raw/archive/features/features'
all_files = os.listdir(features_dir)
csv_files = [f for f in all_files if f.endswith('.csv')]
print(f"File CSV individuati nella cartella features: {len(csv_files)}")

def get_track_id(filename):
    """Converte il nome file nel formato Track_ID standard (es. 2.csv -> T0002)"""
    number = filename.replace('.csv', '')
    return f"T{int(number):04d}"

# Carica il dataframe delle emozioni per verificare la corrispondenza
df_tags = pd.read_csv('../data/raw/deam_emotion_tags.csv')
print(f"Canzoni nel dataset emozioni: {len(df_tags)}")
print(f"Emozioni uniche: {df_tags['Emotion_Label'].unique().tolist()}")

# Prendi l'header del primo file per identificare le features
first_file_path = os.path.join(features_dir, csv_files[0])
first_file_df = pd.read_csv(first_file_path, sep=';', nrows=0)
all_columns = first_file_df.columns
selected_columns = [col for col in all_columns if col != 'frameTime']

print(f"Features audio estratte per canzone: {len(selected_columns)}")
print("   - Feature 'frameTime' esclusa.")
print("   - Tutte le altre colonne (_amean e _stddev) incluse")

# Lista per conservare i dati aggregati di ogni canzone
aggregated_data = []
track_ids_processed = []

print("\nElaborazione dei file in corso...")

for i, file in enumerate(csv_files):
    file_path = os.path.join(features_dir, file)
    track_id = get_track_id(file)
    track_ids_processed.append(track_id)
    
    try:
        # Carica e processa il file
        df_song = pd.read_csv(file_path, sep=';', usecols=selected_columns)
        song_features = df_song.mean().to_dict()
        song_features['Track_ID'] = track_id
        aggregated_data.append(song_features)
        
        # Progress indicator
        if (i + 1) % 200 == 0:
            print(f"   Processate {i + 1} canzoni...")
            
    except Exception as e:
        print(f"   Errore nel file {file}: {e}")
        continue

print("Elaborazione completata!")

# Crea il dataframe finale
df_features = pd.DataFrame(aggregated_data)
print(f"\nDATASET FEATURES CREATO:")
print(f"   - Canzoni processate: {df_features.shape[0]}")
print(f"   - Features per canzone: {df_features.shape[1] - 1}")  # -1 per Track_ID

# ANALISI DI CORRISPONDENZA
print(f"\nANALISI DI CORRISPONDENZA:")
track_ids_tags = set(df_tags['Track_ID'])
track_ids_features = set(df_features['Track_ID'])

matched = track_ids_tags.intersection(track_ids_features)
only_in_tags = track_ids_tags - track_ids_features
only_in_features = track_ids_features - track_ids_tags

print(f"   • Canzoni abbinate: {len(matched)}/{len(df_tags)} ({len(matched)/len(df_tags)*100:.1f}%)")
print(f"   • Canzoni solo in emozioni: {len(only_in_tags)}")
print(f"   • Canzoni solo in features: {len(only_in_features)}")

if only_in_tags:
    print(f"   Track_ID mancanti nelle features: {list(only_in_tags)[:5]}...")

# Unisci i dataset
df_main = df_tags.merge(df_features, on='Track_ID', how='inner')
print(f"\nDATASET FINALE PER MACHINE LEARNING:")
print(f"   - Canzoni totali: {df_main.shape[0]}")
print(f"   - Features totali: {df_main.shape[1] - 4}")  # -4 per colonne metadati
print(f"   - Colonne totali: {df_main.shape[1]}")

# CONTROLLO QUALITÀ
print(f"\nCONTROLLO QUALITÀ DATASET:")
print(f"   • Valori nulli totali: {df_main.isnull().sum().sum()}")
print(f"   • Duplicati: {df_main.duplicated().sum()}")

# Distribuzione emozioni nel dataset finale
emotion_dist = df_main['Emotion_Label'].value_counts()
print(f"\nDISTRIBUZIONE EMOZIONI NEL DATASET FINALE:")
for emotion, count in emotion_dist.items():
    percentage = (count / len(df_main)) * 100
    print(f"   • {emotion}: {count} canzoni ({percentage:.1f}%)")

# Anteprima strutturata
print(f"\nANTEPRIMA DATASET (prime 3 canzoni):")
preview_cols = ['Track_ID', 'Title', 'Valence', 'Arousal', 'Emotion_Label'] + selected_columns[:3]
print(df_main[preview_cols].head(3).to_string(index=False))

# Salva il dataset
output_path = '../data/processed/audio_emotion_dataset.csv'
df_main.to_csv(output_path, index=False)
print(f"\nDATASET SALVATO IN: {output_path}")

print(f"\nPROCESSO COMPLETATO CON SUCCESSO!")
print("=" * 50)

=== ELABORAZIONE FEATURES AUDIO - DATASET DEAM ===
File CSV individuati nella cartella features: 1802
Canzoni nel dataset emozioni: 1802
Emozioni uniche: ['Sad', 'Relaxed', 'Happy', 'Angry']
Features audio estratte per canzone: 260
   - Features per frame esclusa: 'frameTime'
   - Tutte le altre colonne (_amean e _stddev) incluse

Elaborazione dei file in corso...
   Processate 200 canzoni...
   Processate 400 canzoni...
   Processate 600 canzoni...
   Processate 800 canzoni...
   Processate 1000 canzoni...
   Processate 1200 canzoni...
   Processate 1400 canzoni...
   Processate 1600 canzoni...
   Processate 1800 canzoni...
Elaborazione completata!

DATASET FEATURES CREATO:
   - Canzoni processate: 1802
   - Features per canzone: 260

🔍 ANALISI DI CORRISPONDENZA:
   • Canzoni abbinate: 1546/1802 (85.8%)
   • Canzoni solo in emozioni: 256
   • Canzoni solo in features: 256
   ⚠️  Track_ID mancanti nelle features: ['T0783', 'T0141', 'T0066', 'T0061', 'T0567']...

DATASET FINALE PER MACH